In [1]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import pandas as pd
import csv

In [2]:
with open(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles/MScThesisCode/FLP/output/nearestNodePAL.csv', newline='') as f:
    reader = csv.reader(f)
    palFields = list(reader)

origs = list(set(palFields[0]))

In [3]:
for i in range(0, len(origs)):
    origs[i] = int(origs[i])

In [4]:
with open(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles/MScThesisCode/FLP/output/canNetwork.csv', newline='') as f:
    reader = csv.reader(f)
    candidates = list(reader)

dests = list(set(candidates[0]))

In [5]:
for i in range(0, len(dests)):
    dests[i] = int(dests[i])

In [6]:
origs = pd.DataFrame(origs, columns=['origs'])
dests = pd.DataFrame(dests, columns=['dests'])

In [8]:
dests

,dests
0,5266131897
1,5887755359
2,3677820110
3,5430146018
4,6028452163
...,...
480,4840807563
481,5187510896
482,5029857026
483,1835726430


#### Create a dataframe with all combinations of origin-candidates

In [9]:
matrix = origs.merge(dests, how='cross')
print('We have ' + str(len(matrix)) + ' paths')

We have 565510 paths


In [10]:
matrix.dtypes

origs    int64
dests    int64
dtype: object

In [17]:
sampl_df = matrix.head(1000).copy()
sampl_df

,origs,dests
0,5969612910,184096616
1,5969612910,5199178045
2,5969612910,5133301205
3,5969612910,2323332089
4,5969612910,5887755321
...,...,...
995,3273998410,3940146605
996,3273998410,1269591543
997,3273998410,2922161517
998,3273998410,9241311391


In [12]:
# create network from OSMnx
G = ox.graph.graph_from_bbox(11.2, 10.00, -83, -85.2, network_type = 'drive_service')
Gs = ox.utils_graph.get_largest_component(G, strongly=True) #for strongly connected network

In [18]:
%%time
# try to get nearest node with networkx
def short_path_length(row):
    return nx.shortest_path_length(Gs, row['origs'], row['dests'], weight='length')
sampl_df['path_length'] = sampl_df.apply(short_path_length, axis=1)

CPU times: total: 4min 33s
Wall time: 5min 26s


In [20]:
%%time
# try with OSMnx
sampl_df['path_lengthOX'] = ox.shortest_path(Gs, sampl_df['origs'], sampl_df['dests'], weight='length')

CPU times: total: 4min 17s
Wall time: 4min 42s


In [19]:
sampl_df

,origs,dests,path_length
0,5969612910,184096616,39695.028
1,5969612910,5199178045,48125.214
2,5969612910,5133301205,75554.655
3,5969612910,2323332089,159362.941
4,5969612910,5887755321,81660.828
...,...,...,...
995,3273998410,3940146605,88938.535
996,3273998410,1269591543,83789.749
997,3273998410,2922161517,135599.638
998,3273998410,9241311391,73693.783
